# Como usar o `Pipeline`

#### Tags

* Autor: AH Uyekita
* Data: 23/03/22
* Post: [Como usar o `Pipeline`][post_url]

[post_url]: https://andersonuyekita.github.io/notebooks/blog/como-usar-o-pipepline/

#### Descrição

Esse é um arquivo de acompanhamento do _post_ de mesmo nome.

***

### Índice
- [1. Kernel](#1)
- [2. Importação dos _Packages_](#2)
- [3. _Datasets_](#3)
- [4. _Classifiers_](#4)
- [5. Análise dos Resultados](#5)
- [6. _Cross Validation_](#6)
- [7. _Scoring_](#7)
- [8. Técnicas Avançadas](#8)
- [9. Conclusões](#9)
- [10. Versões dos Packages](#10)

***

### 1. Kernel <a id='1'></a>

Foi usado o Python 3.7, conforme pode ser confirmado abaixo.